In [106]:
import pandas as pd
import sqlite3
import os
import json
import sqlalchemy

In [107]:
class a_brick:
    __slots__ = ['_idNum', '_title','_desc','_code','_tags']
    
    def __init__ (self, idNum, title, desc, code, tags):
        self._idNum = idNum
        self._title = str(title)
        self._desc = str(desc)
        self._code = str(code)
        self._tags = str(tags)
    
    @property
    def idNum(self):
        return self._idNum
    
    @property
    def title(self):
        return self._title
    
    @property
    def desc(self):
        return self._desc
    
    @property
    def code(self):
        return self._code
    
    @property
    def tags(self):
        return self._tags
    
    @property
    def astuple(self):
        return (self.idNum, self.title, self.desc, self.code, self.tags)

In [108]:
rootdir = "../finalbricks/"
files = []
output = []

for filename in os.listdir(rootdir):
    if filename.endswith(".ipynb"): 
        files.append(filename)
    else:
        continue

#open each notebook in JSON format
for name in files:
    with open(rootdir+name) as json_file:
        code = json.load(json_file)
        nextfile = ()
        
        for item in code["cells"][0]["source"][1:4]: # we want title, description, and tags only
            item = item.replace('\"','').strip()
            index = item.find(':')
            nextfile += (item[index+2:],)
            
        title, desc, tags = nextfile
        # figure out how to fix the tags later
        brick = a_brick(None, title, desc, code, tags)
        output.append(brick.astuple)


In [109]:
df = pd.DataFrame(output, columns=['id', 'title', 'desc', 'code', 'tags'])

In [117]:
sqlite_db = './bricks3.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

In [123]:
c.execute('CREATE TABLE bricks3 (id INTEGER PRIMARY KEY not None, title TEXT, desc TEXT, code LONGTEXT, tags TEXT);')
conn.commit()

OperationalError: table bricks3 already exists

In [124]:
df.to_sql('bricks3',             # Name of the table
            con=conn,                    # The handle to the file that is setup
            if_exists='replace',         # Overwrite, append, or fail
            index=False)                 # Add index as column

In [125]:
#b1 = a_brick(None, "sup", "desc2", "heres some more code", "python, pandas")


In [126]:
#c.execute('INSERT INTO bricks2 VALUES (?,?,?,?,?)', b1.astuple)
#conn.commit()

In [127]:
results = c.execute('SELECT * from bricks3')
results.fetchall()

[(None,
  'Pandas select rows containing values',
  'Pandas select rows containing values',
  '{\'cells\': [{\'cell_type\': \'markdown\', \'metadata\': {}, \'source\': [\'---\\n\', \'title: "Pandas select rows containing values"\\n\', \'description: "Pandas select rows containing values"\\n\', \'tags: Pandas, EDA\\n\', \'URL:\\n\', \'Licence:\\n\', \'\\n\', \'---\']}, {\'cell_type\': \'markdown\', \'metadata\': {}, \'source\': [\'# Select Rows Containing Values in Pandas\']}, {\'cell_type\': \'code\', \'execution_count\': None, \'metadata\': {}, \'outputs\': [], \'source\': ["# to select rows containing \'keyword\' in column \'my_column\'\\n", "df[df[\'my_column\'].map(lambda x: \'keyword\' in x)]"]}, {\'cell_type\': \'markdown\', \'metadata\': {}, \'source\': [\'---\']}, {\'cell_type\': \'markdown\', \'metadata\': {}, \'source\': [\'## Example\']}, {\'cell_type\': \'markdown\', \'metadata\': {}, \'source\': [\'### Preliminaries\']}, {\'cell_type\': \'code\', \'execution_count\': 12, \

In [49]:
c.execute('DELETE FROM bricks2 WHERE id < 100;')
conn.commit()
results = c.execute('SELECT * from bricks2')
results.fetchall()

[]

In [51]:
for o in output:
    c.execute('INSERT INTO bricks2 VALUES (?, ?, ?, ?, ?)', o)

conn.commit()    
results = c.execute('SELECT * from bricks2')
results.fetchall()

InterfaceError: Error binding parameter 3 - probably unsupported type.